In [1]:
from google.colab import drive
drive.mount('/content/drive')



import pandas as pd
import numpy as np

#df = pd.read_excel("/content/drive/MyDrive/Tanjila_mam/DS-Healthcare_version_2.xlsx")

!pip install -q flwr["simulation"] tensorflow

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.1/465.1 kB 22.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.5 which is incompatible.


In [ ]:
import math
from typing import Dict, List, Tuple
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    f1_score,
    recall_score,
    precision_score,
    roc_auc_score,
)
import flwr as fl
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from flwr.common import Metrics
from flwr.simulation.ray_transport.utils import enable_tf_gpu_growth


import math
from typing import Dict, List, Tuple
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from flwr.common import Metrics
from sklearn.metrics import (
    confusion_matrix,
    f1_score,
    recall_score,
    precision_score,
    roc_auc_score,
)
import flwr as fl
import numpy as np

from tqdm import tqdm
import matplotlib.pyplot as plt



num_of_clients = [10,20,30,40,50]


for NUM_CLIENTS in tqdm(num_of_clients):

  df = pd.read_excel("/content/drive/MyDrive/Tanjila_mam/DS-Healthcare_version_2.xlsx")

  VERBOSE = 0
  #NUM_CLIENTS = 30
  count_rounds = 0
  number_of_rounds = 15
  saving_model_name = ""
  metrics_dict = {"client": [], "centralized": []}  # Dictionary to store metrics


  # ANN Model
  def get_model():
      model = tf.keras.models.Sequential(
          [
              tf.keras.layers.Flatten(input_shape=[9]),
              tf.keras.layers.Dense(128, activation="relu"),
              tf.keras.layers.Dropout(0.2),
              tf.keras.layers.Dense(3, activation="softmax"),
          ]
      )
      global saving_model_name, NUM_CLIENTS, number_of_rounds
      saving_model_name = str(NUM_CLIENTS) + "_" + str(number_of_rounds) + "_ann"
      model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
      return model


  # Flower Client
  class FlowerClient(fl.client.NumPyClient):
      def __init__(self, x_train, y_train, x_val, y_val) -> None:
          self.model = get_model()
          self.x_train, self.y_train = x_train, y_train
          self.x_val, self.y_val = x_val, y_val

      def get_parameters(self, config):
          return self.model.get_weights()

      def fit(self, parameters, config):
          self.model.set_weights(parameters)
          history = self.model.fit(
              self.x_train, self.y_train, epochs=1, batch_size=32, verbose=VERBOSE
          )
          train_loss = history.history["loss"][-1]
          train_acc = history.history["accuracy"][-1]

          return self.model.get_weights(), len(self.x_train), {
              "train_loss": train_loss,
              "train_accuracy": train_acc,
          }

      def evaluate(self, parameters, config):
          self.model.set_weights(parameters)
          y_pred = np.argmax(self.model.predict(self.x_val, verbose=VERBOSE), axis=1)
          loss, acc = self.model.evaluate(self.x_val, self.y_val, batch_size=64, verbose=VERBOSE)

          # Confusion Matrix and Metrics
          cm = confusion_matrix(self.y_val, y_pred)
          f1 = f1_score(self.y_val, y_pred, average="weighted")
          precision = precision_score(self.y_val, y_pred, average="weighted")
          recall = recall_score(self.y_val, y_pred, average="weighted")
          auc = roc_auc_score(self.y_val, self.model.predict(self.x_val), multi_class="ovr")

          misclassification_rate = 1 - acc
          fpr = cm[0][1] / (cm[0][1] + cm[0][0])
          fnr = cm[1][0] / (cm[1][0] + cm[1][1])

          # Flatten confusion matrix to avoid passing list of lists
          flat_confusion_matrix = np.array(cm).flatten().tolist()

          metrics = {
              "test_accuracy": acc,
              "test_loss": loss,
              "f1_score": f1,
              "precision": precision,
              "recall": recall,
              "auc": auc,
              "misclassification_rate": misclassification_rate,
              "fpr": fpr,
              "fnr": fnr,
              "confusion_matrix": flat_confusion_matrix,
          }

          metrics_dict["client"].append(metrics)  # Store client metrics

          return loss, len(self.x_val), metrics


  # Partition Dataset
  def partition_dataset():
      encoder = LabelEncoder()
      df['Type'] = encoder.fit_transform(df['Type'])
      X = df.iloc[:, 4:]
      y = df['Type']

      # Apply SMOTE to balance the dataset
      smote = SMOTE(random_state=42)
      X_resampled, y_resampled = smote.fit_resample(X, y)

      x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.33, random_state=0)

      x_train = x_train.to_numpy()
      x_test = x_test.to_numpy()
      y_train = y_train.to_numpy()
      y_test = y_test.to_numpy()

      testset = (x_test, y_test)
      partitions = []
      partition_size = math.floor(len(x_train) / NUM_CLIENTS)
      for cid in range(NUM_CLIENTS):
          idx_from, idx_to = int(cid) * partition_size, (int(cid) + 1) * partition_size
          partitions.append((x_train[idx_from:idx_to], y_train[idx_from:idx_to]))
      return partitions, testset


  # Client function
  def get_client_fn(dataset_partitions):
      def client_fn(cid: str) -> fl.client.Client:
          x_train, y_train = dataset_partitions[int(cid)]
          split_idx = math.floor(len(x_train) * 0.9)
          x_train_cid, y_train_cid = x_train[:split_idx], y_train[:split_idx]
          x_val_cid, y_val_cid = x_train[split_idx:], y_train[split_idx:]
          return FlowerClient(x_train_cid, y_train_cid, x_val_cid, y_val_cid)
      return client_fn


  # Centralized (Global) evaluation function
  def get_evaluate_fn(testset):
      x_test, y_test = testset

      def evaluate(server_round: int, parameters: fl.common.NDArrays, config: Dict[str, fl.common.Scalar]):
          model = get_model()
          model.set_weights(parameters)
          y_pred = np.argmax(model.predict(x_test, verbose=VERBOSE), axis=1)
          loss, accuracy = model.evaluate(x_test, y_test, verbose=VERBOSE)

          # Confusion Matrix and Metrics
          cm = confusion_matrix(y_test, y_pred)
          f1 = f1_score(y_test, y_pred, average="weighted")
          precision = precision_score(y_test, y_pred, average="weighted")
          recall = recall_score(y_test, y_pred, average="weighted")
          auc = roc_auc_score(y_test, model.predict(x_test), multi_class="ovr")

          misclassification_rate = 1 - accuracy
          fpr = cm[0][1] / (cm[0][1] + cm[0][0])
          fnr = cm[1][0] / (cm[1][0] + cm[1][1])

          # Flatten confusion matrix
          flat_confusion_matrix = np.array(cm).flatten().tolist()

          global count_rounds
          global number_of_rounds

          if number_of_rounds - count_rounds == 0:
              global saving_model_name
              model.save(saving_model_name + "_model.h5")

          count_rounds += 1

          metrics = {
              "test_accuracy": accuracy,
              "test_loss": loss,
              "f1_score": f1,
              "precision": precision,
              "recall": recall,
              "auc": auc,
              "misclassification_rate": misclassification_rate,
              "fpr": fpr,
              "fnr": fnr,
              "confusion_matrix": flat_confusion_matrix,
          }

          metrics_dict["centralized"].append(metrics)  # Store centralized metrics

          return loss, {
              "accuracy": accuracy,
              "loss": loss,
              "f1_score": f1,
              "precision": precision,
              "recall": recall,
              "auc": auc,
              "misclassification_rate": misclassification_rate,
              "fpr": fpr,
              "fnr": fnr,
              "confusion_matrix": flat_confusion_matrix,
          }

      return evaluate


  # Aggregation function for metrics (weighted average)
  def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
      accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
      losses = [num_examples * m["loss"] for num_examples, m in metrics]
      f1_scores = [num_examples * m["f1_score"] for num_examples, m in metrics]
      precisions = [num_examples * m["precision"] for num_examples, m in metrics]
      recalls = [num_examples * m["recall"] for num_examples, m in metrics]
      aucs = [num_examples * m["auc"] for num_examples, m in metrics]
      misclassification_rates = [num_examples * m["misclassification_rate"] for num_examples, m in metrics]
      fprs = [num_examples * m["fpr"] for num_examples, m in metrics]
      fnrs = [num_examples * m["fnr"] for num_examples, m in metrics]
      cm = [m["confusion_matrix"] for _, m in metrics]

      total_examples = sum([num_examples for num_examples, _ in metrics])

      return {
          "accuracy": sum(accuracies) / total_examples,
          "loss": sum(losses) / total_examples,
          "f1_score": sum(f1_scores) / total_examples,
          "precision": sum(precisions) / total_examples,
          "recall": sum(recalls) / total_examples,
          "auc": sum(aucs) / total_examples,
          "misclassification_rate": sum(misclassification_rates) / total_examples,
          "fpr": sum(fprs) / total_examples,
          "fnr": sum(fnrs) / total_examples,
          "confusion_matrix": np.mean(cm, axis=0).flatten().tolist(),
      }



  # Partition data and start Flower simulation
  dataset_partitions, testset = partition_dataset()
  client_fn = get_client_fn(dataset_partitions)
  evaluate_fn = get_evaluate_fn(testset)

  # Start FL Simulation
  fl.simulation.start_simulation(
      client_fn=client_fn,
      num_clients=NUM_CLIENTS,
      config=fl.server.ServerConfig(num_rounds=number_of_rounds),
      strategy=fl.server.strategy.FedAvg(
          evaluate_fn=evaluate_fn,
          fraction_fit=0.1,
          fraction_evaluate=0.1,
          min_fit_clients=NUM_CLIENTS,
          min_evaluate_clients=NUM_CLIENTS,
          min_available_clients=NUM_CLIENTS,
          on_fit_config_fn=lambda rnd: {"rnd": rnd},
          on_evaluate_config_fn=lambda rnd: {"rnd": rnd},
          evaluate_metrics_aggregation_fn=weighted_average,
      ),
  )






  centralized_metrics = {
      'test_accuracy': [],
      'test_loss': [],
      'f1_score': [],
      'precision': [],
      'recall': [],
      'auc': [],
      'misclassification_rate': [],
      'fpr': [],
      'fnr': [],
      'confusion_matrix': []
  }

  # Extract the values for each metric from each centralized result
  for entry in metrics_dict['centralized']:
      for key in centralized_metrics.keys():
          centralized_metrics[key].append(entry[key])

  # Print the reformatted dictionary
  print(centralized_metrics)





  import json

  # Path to your JSON file
  json_file_path = '/content/drive/MyDrive/Tanjila_mam/JSON Result/ANN_json_file_number_of_client.json'

  # Load the existing JSON file
  try:
      with open(json_file_path, 'r') as file:
          data = json.load(file)
  except FileNotFoundError:
      # If the file doesn't exist, initialize with an empty structure
      data = {}

  # Ensure 'centralized_metrics' is a list
  if 'centralized_metrics' not in data:
      data['centralized_metrics'] = []

  # Append the new centralized metrics
  if isinstance(data['centralized_metrics'], list):
      data['centralized_metrics'].append(centralized_metrics)
  else:
      # If 'centralized_metrics' is not a list, convert it to a list and append the new data
      data['centralized_metrics'] = [data['centralized_metrics'], centralized_metrics]

  # Write the updated data back to the JSON file
  with open(json_file_path, 'w') as file:
      json.dump(data, file, indent=4)
  print("___________________________________________________________________________________________________________________________________")
  print("___________________________________________________________________________________________________________________________________")
  print("New centralized metrics appended and JSON file updated successfully for Clients:",NUM_CLIENTS," Number of Rouns: ",number_of_rounds)



Output hidden; open in https://colab.research.google.com to view.

In [ ]:
centralized_metrics = {
    'test_accuracy': [],
    'test_loss': [],
    'f1_score': [],
    'precision': [],
    'recall': [],
    'auc': [],
    'misclassification_rate': [],
    'fpr': [],
    'fnr': [],
    'confusion_matrix': []
}

# Extract the values for each metric from each centralized result
for entry in metrics_dict['centralized']:
    for key in centralized_metrics.keys():
        centralized_metrics[key].append(entry[key])

# Print the reformatted dictionary
print(centralized_metrics)

{'test_accuracy': [0.3337828814983368, 0.8500904440879822, 0.8740978240966797, 0.8773615956306458, 0.8871893882751465, 0.8986856341362], 'test_loss': [11.377601623535156, 0.4172004461288452, 0.3452754020690918, 0.3227986991405487, 0.31076815724372864, 0.29018548130989075], 'f1_score': [0.16706018188578928, 0.8485856087604265, 0.8730831705416685, 0.8759081218145705, 0.8860897207083667, 0.8978690808875907], 'precision': [0.11141100444687232, 0.8600342045122951, 0.8798852430192183, 0.8858205984066837, 0.8932858690436479, 0.9029220979020873], 'recall': [0.3337828702118674, 0.8500904577108029, 0.8740978125048878, 0.8773615892750697, 0.8871894154919909, 0.8986856260003788], 'auc': [0.5277273309664025, 0.9556364653292508, 0.9708721822200772, 0.9758055055394955, 0.9772346008519502, 0.9800351615694165], 'misclassification_rate': [0.6662171185016632, 0.14990955591201782, 0.1259021759033203, 0.12263840436935425, 0.11281061172485352, 0.10131436586380005], 'fpr': [0.0, 0.057055947388996886, 0.09216

In [ ]:
import json

# Path to your JSON file
json_file_path = 'your_json_file.json'

# New centralized_metrics data
new_centralized_metrics = {
    'test_accuracy': [0.3337828814983368,  0.8871893882751465, 0.8986856341362],
    'test_loss': [ 0.31076815724372864, 0.29018548130989075],
    'f1_score': [0.16706018188578928,0.8978690808875907],
    'precision': [0.11141100444687232,  0.9029220979020873],
    'recall': [0.3337828702118674, 0.8986856260003788],
    'auc': [0.5277273309664025,  0.9800351615694165],
    'misclassification_rate': [0.6662171185016632, 0.11281061172485352, 0.10131436586380005],
    'fpr': [0.0, 0.052070518512179205],
    'fnr': [1.0, 0.024057826781552, 0.028134820124226214],
    'confusion_matrix': [
        [192061, 191415, 0, 0],
        [176075, 10654, 5332, 14407, 174185, 3339, 31166, 21361, 138888],
        [167408, 16996, 7657, 5438, 185167, 1326, 22178, 18850, 150387],
        [171177, 16034, 4850, 3824, 187208, 899, 23551, 21409, 146455],
        [173205, 13172, 5684, 4593, 186322, 1016, 20492, 19955, 150968],
        [177223, 9735, 5103, 5354, 184944, 1633, 23253, 13219, 154943]
    ]
}

# Load the existing JSON file
try:
    with open(json_file_path, 'r') as file:
        data = json.load(file)
except FileNotFoundError:
    # If the file doesn't exist, initialize with an empty structure
    data = {}

# Ensure 'centralized_metrics' is a list
if 'centralized_metrics' not in data:
    data['centralized_metrics'] = []

# Append the new centralized metrics
if isinstance(data['centralized_metrics'], list):
    data['centralized_metrics'].append(new_centralized_metrics)
else:
    # If 'centralized_metrics' is not a list, convert it to a list and append the new data
    data['centralized_metrics'] = [data['centralized_metrics'], new_centralized_metrics]

# Write the updated data back to the JSON file
with open(json_file_path, 'w') as file:
    json.dump(data, file, indent=4)

print("New centralized metrics appended and JSON file updated successfully!")


New centralized metrics appended and JSON file updated successfully!
